In [19]:
import numpy as np
import pandas as pd
import yfinance as yf

# Finance
## Risk Vs. Return
Why would anyone invest money instead of keeping the money at home in a safe? it is because we want to grow our money. Why do we want to grow our money? There is a need and greed to do so - The need is that, inflation will render our money less valuable in the future. So we have to grow our money at least as much as inflation devalues our money. The greed is self explanatory. In this section, our entire focus will be on how much growth one expects from an investment - We will start with the simplest of markets, where there is only one security in which we can invest, and introduce more and more securities to see how our decision would change in each case.

### Single risk-free bond
Suppose the market only has one security - it is a riskless government bond, which offers \\$1 after one year. How much should we pay for this bond? Suppose inflation rate is 2\% yearly. Then it makes sense that, at a minimum, if we stick \\$x into a risk-free bond, we should at least get \\$x\*1.02. The reason is, if it doesn't pay at least this much, we would just buy all the non-perishable items in the CPI basket with \\$x and keep it home and sell them after 1 year and make \\$x\*(1.02). So if the bond face value is \\$1, then we cannot pay more than $ \frac{\$1}{1.02} = 0.9803*\$1 = \$0.9803 $. The multiplicative factor 0.9803 in the above calculation is called the **discount factor** - It is the factor by which we discount a future dollar. One can think of it as a way of converting a future dollar into today's dollar (just like we would use a conversion factor to go between two different currencies). If we want to converse in terms of the denominator, the value in excess of 1 is called the **rate of return** or simply **return** - i.e., 


$$ r = \frac{\$1}{\$0.9803} - 1 = \frac{\$1 - \$0.9803}{\$0.9803} $$ 


So the return is essentially profit divided by invested amount[<sup>[\*]</sup>](#fn1).<a name = "bffn1"></a>

<div class=highlight_box>
U.S. treasury bonds, considered riskfree typically trade at a only few basis points above the inflation.
</div>
</br>
<span id="fn1"> *We are assuming that there won't be any tax on the profit. This isn't necessarily true. But for pedagogical reasons, we will assume that there is no tax in any investment. Also we are assuming that there won't be any warehousing cost if we want to buy instead a basket of CPI goods and store them. If this is not true, the returns we demand from a riskfree bond may be even lower than the inflation rate. [Go back](#bffn1)</span>

In [20]:
df_list = pd.read_html("https://finance.yahoo.com/bonds/")
tyields = df_list[0]
tyields[['Symbol', 'Name', 'Last Price']]

,Symbol,Name,Last Price
0,^IRX,13 Week Treasury Bill,0.093
1,^FVX,Treasury Yield 5 Years,0.339
2,^TNX,Treasury Yield 10 Years,0.774
3,^TYX,Treasury Yield 30 Years,1.562


### One risk-free bond and one risky bond
Now, let us say that we have a choice between a riskfree government bond and a corporate bond that is risky - risky in that, there is a chance of default. Let us say that on a \\$1 investment in this bond, there is a 50\% chance of default where we get \\$0 in return after one year and a 50\% chance that we will get \\$2x. This means the expected value of this investment is $ 0.5*\$0 + 0.5*\$2x = \$x $. The question is what should be the value of \\$x that will make us invest in the corporate bond instead of the riskfree government bond? Since \\$1 invested in a government bond will give us \\$1.02, \\$x must be greater than \\$1.02. How much greater? The answer is entirely a personal choice - And hence it will vary from one person to another. Let us call the return from this risky bond as $ R_i = \frac{\$x}{\$1} - 1 $. Suppose we call the risk free return of the government bond, $ r_f $, we have: 

$$ E[R_i] = r_f + risk\ premium $$
    
where, the risk premium is the excess return on top of riskless return that we need in order to invest in the corporate bond

In the wake of that question, we can make the following additional observations:

- The probabilty of default is a prediction. And the prediction may be right or wrong. So the risk premium must contain within it, an uncertainty premium \- because there is any uncertainty involved at all as compared to a risk-free bond \- and also a prediction premium because we are taking a risk in making a prediction of the probabilities. In other words, there is a probability that your probability estimates are wrong
- Other people in the market may estimate the chances of default differently than us, besides the fact that they may want a demand a different risk premium than we do. 
- The price of a security, whether or not it is risky, will depend entirely upon supply-demand. So the price may be higher than what we demand or lower. Even though the prices are fixed by the market, our decision to invest or not should depend only on our appetite for taking risks and the returns that we want. There isn't really such a thing as a ``fair price'' for a security. 
- The predictions about chances of default can be made only with available data - i.e., present and past data. We cannot predict based on future data
    
An important note before we move on: So far, we have considered government bonds to be risk free. But the truth is, the risk free return is a misnomer because it is just a rate the market collectively demanded as something greater than the rate of inflation - And the rate of inflation itself is a prediction (although very accurate). This is the reason why the price of government bonds (which is a function of only r_f as the cash flows are constants) varies from time to time. If it was truly risk free, that wouldn't be the case!

### One risk-free bond and two risky bonds
Suppose we have two bonds, A and B, such that, A will pay \\$2 or \\$0, each with a probability of 0.5, while B will pay \\$1.5 or \\$0.5, each with a probability of 0.5. Both bond A and bond B have the same expected payoff, but are otherwise different in their payoff-probability structure. Would we pay the same price for both bonds? It doesn't sound right to pay the same price for both bonds. Now consider another scenario: bonds A and B have the same payoffs, namely \\$2 or \\$0, but with different probabilities - would we pay the same price for both? Again, it doesn't sound right to do so. In this section, we will develop a way to quantify risk and a way to compute the fair price for A given the price for B or vice-versa: Note that, we are going to come up a comparative pricing model, not an absolute pricing model - We are arguing that, while how much to pay for a given risk level may be a personal preference, once the preference is set, how one values two risky assets has to be consistent. 

In our examples of risky bonds, we have considered, so far, only two payoff possibilities - one when there is a default and one when there isn't - i.e., the bond payoffs are a Bernoulli random variable. In reality, it will be a much more complex random variable with several payoff possibilities and probabilities associated with them. There are many reasons for this:
1. The payoff when the company defaults isn't going to be a single value. When a default happens, creditors take control of the company and liquidate assets. Then depending on whether the bond is serior or junior, we will get a payoff proportional to our debt ownership within our bond category. So when we predict this payoff, we can think of several possible scenarios each of which will mean different payoffs for us - And these scenarios will all come with their own probabilities
2. The payoff need not always come from the company that issued the bond. If the bond is a security, it can be traded in the market for a higher price that what we got it for. But this means thinking about several kinds of risks starting with market risk, all of which can affect the price of the bond - and hence the returns - in different ways

Besides these reasons, even if the bond is indeed a Bernoulli random variable, the only way to meaningfully talk about expected payoffs and returns is if we were to make several observations of this bond - i.e., we toss the coin enough times for the idea of probabilistic outcomes to make sense. So, for any probability based analysis of these bond to make sense, we have to buy the same bond, year-after-year, for many many years and then take a sample average of their returns. As we know from the central limit theorem, we will end up with normal random variables for the average return of both bonds - So we will have to deal with a random variable that has various payoff possibilites and associated probabilities than the simple case of Bernoulli random variables. In this section, for the sake of deriving a meaningful risk vs. return criterion, we will consider the bonds' payoffs as normal random variables.

Say, bond A's payoff is $ \sim \mathcal{N}(\mu_A,\sigma^2_A) $ and bond B's payoff is $ \sim \mathcal{N}(\mu_B,\sigma^2_B) $. To come up with a comparitive pricing model, we need a criterion: Let us say our criterion is that, **we want to be 75\% confident that the payoff isn't less than a target return**. Let us call this target return as $r_t$. If the price of bond A is $P_A$, then our criterion translates to the payoff of bond A being greater than or equal to $P_A(1+r_t)$ with a probability of 0.75 and the payoff of bond B being greater than or equal to $P_B(1+r_t)$ with a probability of 0.75. For notational convenience, let us call $P_A(1+r_t)$ as $T_A$ and $P_B(1+r_t)$ as $T_B$. Then our criterion can be mathematically written as:

\begin{align}
\frac{1}{\sigma_A\sqrt{2\pi}} \int_{T_A}^{\infty} e^{-\frac{1}{2}\left(\frac{x - \mu_A}{\sigma_A}\right)^2} dx\ &=\ \frac{1}{\sigma_B\sqrt{2\pi}} \int_{T_B}^{\infty} e^{-\frac{1}{2}\left(\frac{x - \mu_B}{\sigma_B}\right)^2} dx\ =\ 0.75 \\
\\
1 - CDF_{\mu_A, \sigma_A}(T_A)\ &=\ 1 - CDF_{\mu_A, \sigma_A}(T_B)\\
\\
1 - \frac{1}{2}\left[1+\operatorname{erf}\left({\frac{x-\mu_A}{\sigma_A{\sqrt{2}}}}\right)\right]^{T_A}_{-\infty}\ &=\ 1 - \frac{1}{2}\left[1+\operatorname{erf}\left({\frac{x-\mu_B}{\sigma_B{\sqrt{2}}}}\right)\right]^{T_B}_{-\infty}
\end{align}

Cancelling out the constants, and realizing that the error function is a strictly increasing function, we have

$$\frac{T_A-\mu_A}{\sigma_A{\sqrt{2}}}\ =\ \frac{T_B-\mu_B}{\sigma_B{\sqrt{2}}} $$

substituting $P_A(1+r_t)$ for $T_A$ and $P_B(1+r_t)$ for $T_B$, multiplying by -1 on both sides, we get},

\begin{equation*}
\frac{\mu_A - P_A(1+r_t)}{\sigma_A}\ =\ \frac{\mu_B - P_B(1+r_t)}{\sigma_B}
\label{eq:bond_value_equality} \tag{1}
\end{equation*}

Note that, while our confidence level of 75\% will dictate the exact values of the L.H.S. and the R.H.S. of the above equation, the equality in the above equation holds true no matter what the confidence level is. 

The actual rate of returns of bond A and bond B are random variable, which is a translated version of their corresponding payoff R.V.'s. So we can calculate their expected values as:

$$ E[R]_\omega = \frac{\mu_\omega - P_\omega}{P_\omega}\ |\ \omega\ =\ A\ or\ B $$

i.e., 

$$ \mu_\omega = P_\omega(1+E[R]_\omega)\ |\ \omega\ =\ A\ or\ B $$

Using this, we can rewrite $\eqref{eq:bond_value_equality}$ in terms of only the bond prices and their expected returns as:

\begin{equation*}
\frac{P_A(1+E[R_A]) - P_A(1+r_t)}{\sigma_A}\ =\ \frac{P_B(1+E[R_B]) - P_B(1+r_t)}{\sigma_B}\\
\\
\frac{P_A}{\sigma_A}(E[R_A]-r_t)\ =\ \frac{P_B}{\sigma_B}(E[R_B]-r_t)\label{eq:price_returns_sigma} \tag{2}
\end{equation*}

Let us explore two scenarios with the above equality.

**Price invariance**: If we want to pay the same price for both bonds, then we can derive the following relationship between the bonds' rate of returns from equation $\eqref{eq:price_returns_sigma}$ as:

\begin{align}
\frac{E[R_A]-r_t}{\sigma_A} &= \frac{E[R_B]-r_t}{\sigma_B}\\
\\
E[R_A] &= r_t + \frac{\sigma_A}{\sigma_B}(E[R_B]-r_t) \label{eq:price_invariance} \tag{3}
\end{align}

Equation $\eqref{eq:price_invariance}$ gives us a concrete understanding of risk vs. return: $\sigma$ is risk and $\frac{E[R]-r_t}{\sigma} $ excess return per unit of risk. A special case of equation $\eqref{eq:price_invariance}$ is if we set $r_t=r_f$, i.e., Suppose we want to pay the same price for both bonds, as we would for a risk-free security, then, we have the following equality:

$$
E[R_A] = r_f + \frac{\sigma_A}{\sigma_B}(E[R_B]-r_f)
$$

One can see that, when $\sigma_A$ is 0, i.e., bond A is basically risk-free, then $E[R_A] = r_f$. 

**Returns invariance**: If we want to get the same return from both bonds, then we can substitute $E[R_A] = E[R_B]$ in equation $\eqref{eq:price_returns_sigma}$ and get:

$$
\frac{P_A}{\sigma_A} = \frac{P_B}{\sigma_B}
$$

Here, one could talk about the price of security per unit of risk. One can think of $\sigma_A/\sigma_B$ as a conversion factor between one price and another, one excess return and another. Before we proceed to the next section, here are some things to keep in mind aboute the above results:

<div class=highlight_box>
<b>Caveats</b>
<ul>
<li>The payoffs are all predicted figures. Predictions can have errors.</li> 
<li>The results are based on one, loss-limiting, criterion. It omits the fact that, there is considerable upsides to taking risks.</li> 
<li>Our pricing methodology need not be the same as that of the other market participants. The "fair price" of the security, as decided by the meeting point of the supply\-demand curve may be above what you think is a fair price. You can simply not invest. But you can't argue whether the price is fair or not.</li>
<li> When talking about risk-free return or target return, it is important to ensure the time periods associated with them are the same as the context in which they are used. For instance, while talking about the comparative pricing of bonds as we have done above, if the bonds in question have a 1-year maturity period, then we may use a 1 year T-note for $r_f$, not a 10 year T-note </li>
<li>Payoffs here are assumed to be what ends up in our hands when the bond matures - i.e., after accounting for transaction costs, taxes etc. If that isn't true, then the results above are also not true</li>
</div>

### Complex market with diversification opportunities
Until now, we have talked about risks and returns associated with single securities. But in complex markets, investors have the opportunity to build a portfolio that contains several securities - so we can talk about the risks and returns of 
Consider two risky bonds, whose payoffs A, B, are random variables. Suppose we put together these bonds into a portfolio - Let us call the payoff of this portfolio as a random variable X. Now we know that,

$$
E[X] = E[A] + E[B]
$$

But what about the standard deviation of X? It will depend on the correlation coefficient $\rho_{AB} = \frac{Cov(A,B)}{\sigma_A\sigma_B}$. Let us look at the following three special cases:
- When $\rho_{AB} = 1$, i.e, A and B are perfectly, positively correlated:
$$ \sigma_X = \sigma_A + \sigma_B$$
- When $\rho_{AB} = 0$, i.e., A and B are independent: 
$$ \sigma_X = \sqrt{\sigma_A+\sigma_B} $$
- When $\rho_{AB} = -1$, i.e, A and B are perfectly, negatively correlated:
$$ \sigma_X = \sigma_A - \sigma_B$$

In the previous section, we saw that the standard deviation is a good way to characterize risk. With that in context, we observe the following points about our portfolio:
- When A and B are independent of each other, the risk is smaller than if A and B were positively correlated
- When A and B are negatively correlated, there is a chance of reducing the risk even lower than if they are independent. In fact, if A and B are perfectly negatively correlated, and are equal to one another, we get $\sigma_X = 0$. Our portfolio becomes a risk-free portfolio - i.e, we created a risk-free security by combining two risky securities! Notice that, no matter how A and B are associated with each other, the expected payoff is always the same.

In [29]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()